In [1]:
#!pip install -r requirements.txt

In [2]:
import yaml
import timeit
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from utility import *
from ultralytics import settings
from registry import get_model, get_pipeline
import os
settings.update({'datasets_dir': os.path.abspath('.')})


Install mish-cuda to speed up training and inference. More importantly, replace the naive Mish with MishCuda will give a ~1.5G memory saving during training.


✅ YOLOv8 통합 모델 로드 성공
✅ YoloOW 통합 모델 로드 성공
✅ FFCA-YOLO 통합 모델 로드 성공
✅ 3개 통합 인터페이스 모델이 로드되었습니다.
📝 총 14개 모델이 registry에 등록되었습니다.


c:\Users\user\anaconda3\envs\AerialObjectDetection\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
c:\Users\user\anaconda3\envs\AerialObjectDetection\lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
C:\Users\user\Desktop\Aerial Object Detection\AERIAL_OBJECT_DETECTION\Models\YOLOH\models\backbone\convnext.py:204: UserWarning: Overwriting convnext_tiny in registry with models.backbone.convnext.convnext_tiny. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def convnext_tiny(pretrained=False, res_dilation=False, **kwargs):
C:\U

In [3]:
project_name = 'Object Detection'
# dataset_root 가 상대경로일 경우 colab 환경에서는 무조건 앞에 datasets 를 붙여서 찾는 문제가 있음
dataset_root = 'Datasets'
output_dir = 'output'
dataset_names = ['NWPU_VHR10_YOLO', 'COCO_airplane',]
model_names = ['MSNet_CLI','FFCA_YOLO_CLI','YoloOW_CLI']
iterations = [1,2]
epochs = 50
early_stop = 20
batch_size = 16
image_size = 640
optimizer = 'AdamW'
lr = 0.001 
momentum = 0.9
weight_decay = 1e-4
optim_args = {'optimizer': optimizer, 'lr': lr, 'momentum': momentum, 'weight_decay': weight_decay}
devices = [0]
device = torch.device("cuda:"+str(devices[0])) if len(devices)>0 else torch.device("cpu")
train_split = 0.6
val_split = 0.2
test_split = 0.2
num_workers = 0

In [ ]:
from utility.utils import control_random_seed, update_dataset_paths, format_measures, merge_and_update_df
# 실험 시작 전에 DEBUG 로거 초기화
from registry import initialize_experiment_logging
from datetime import datetime

# 실험 ID 생성 (예: 250602_114530)
experiment_id = datetime.now().strftime("%y%m%d_%H%M%S")
debug_logger = initialize_experiment_logging(experiment_id)

print(f"🔍 DEBUG 로그는 다음 파일에 저장됩니다:")
if debug_logger:
    print(f"📁 {debug_logger.get_log_file_path()}")

Experiments_Time = datetime.now().strftime("%y%m%d_%H%M%S")
print('Experiment Start Time:',Experiments_Time)

ex_dict = {}
ex_dict['Experiment Time'] = Experiments_Time;ex_dict['Epochs'] = epochs;
ex_dict['Batch Size'] = batch_size;
ex_dict['Early Stop'] = early_stop;
ex_dict['Device'] = device
ex_dict['Optimizer'] = optimizer;
ex_dict['LR']=optim_args['lr']; ex_dict['Weight Decay']=optim_args['weight_decay'];ex_dict['Momentum']=optim_args['momentum'];
#ex_dict['Hyp'] = dict(lr0=ex_dict['LR'], momentum=ex_dict['Momentum'], weight_decay=ex_dict['Weight Decay'])
ex_dict['Image Size'] = image_size
ex_dict['Output Dir'] = output_dir
ex_dict['Num Workers'] = num_workers
ex_dict['AutoAnchor'] = True

for iteration in range(iterations[0], iterations[1]+1):
    print(f'(Iter {iteration})')
    seed = iteration
    ex_dict['Iteration'] = iteration
    for j, Dataset_Name in enumerate(dataset_names):
        print(f'Dataset: {Dataset_Name} ({j+1}/{len(dataset_names)})');
        # 클래스별 메트릭 초기화 - 이전 데이터셋의 클래스별 메트릭이 남아있는 문제 해결
        for key in list(ex_dict.keys()):
            if '/' in key:  # 클래스별 메트릭은 'class_name/metric_name' 형식
                del ex_dict[key]
        control_random_seed(seed)
        data_yaml_path = f"{dataset_root}/{Dataset_Name}/data_iter_{iteration:02d}.yaml"
        with open(data_yaml_path, 'r') as f:
            data_config = yaml.load(f, Loader=yaml.FullLoader)
        ex_dict['Dataset Name'] = Dataset_Name; ex_dict['Data Config'] = data_yaml_path; ex_dict['Number of Classes'] = data_config['nc']; ex_dict['Class Names'] = data_config['names'];
        ex_dict['Split Size'] = f"{data_config['train_count']}/{data_config['val_count']}/{data_config['test_count']}"
        update_dataset_paths(dataset_root, Dataset_Name, iteration)
        
        for k, model_name in enumerate(model_names):
            print(f'{model_name} ({k+1}/{len(model_names)}) (Iter {iteration})', end=' ')
            print(f'Dataset: {Dataset_Name} ({j+1}/{len(dataset_names)})', end=' ')
            control_random_seed(seed)
            
            model = get_model(model_name, ex_dict)
            train_fn, eval_fn, test_fn = get_pipeline(model_name)

            ex_dict['Model Name'] = model_name; ex_dict['Model']=model;
            start = timeit.default_timer()

            ex_dict = train_fn(ex_dict)
            ex_dict = eval_fn(ex_dict)
            ex_dict = test_fn(ex_dict)

            ex_dict['Train-Test Time'] = timeit.default_timer() - start
            #################
            if ex_dict.get('Test Results') is None:
                raise RuntimeError("평가 결과가 없습니다. 위의 체크리스트를 참고해 원인부터 해결하세요.")

            eval_dict = format_measures(ex_dict.get('Test Results'))
            output_csv = f"{ex_dict['Experiment Time']}_{project_name}_Results.csv"
            merge_and_update_df(ex_dict, eval_dict, output_csv, exclude_columns=['Model', 'Train Results', 'Test Results'])

📋 DEBUG 로그 파일: logs\debug_250605_092720.log
🔍 DEBUG 로그는 다음 파일에 저장됩니다:
📁 logs\debug_250605_092720.log
Experiment Start Time: 250605_092720
(Iter 1)
Dataset: NWPU_VHR10_YOLO (1/2)
MSNet_CLI (1/2) (Iter 1) Dataset: NWPU_VHR10_YOLO (1/2) MSNet 학습 명령어: c:\Users\user\anaconda3\envs\AerialObjectDetection\python.exe c:\Users\user\Desktop\Aerial Object Detection\AERIAL_OBJECT_DETECTION\Models\MSNet\SourceFile\train.py --cuda=True --seed=3407 --distributed=False --sync_bn=False --fp16=False --classes_path=c:\Users\user\Desktop\Aerial Object Detection\AERIAL_OBJECT_DETECTION\Models\MSNet\SourceFile\model_data\classes.txt --model_path= --input_shape=[640, 640] --phi=l --pretrained=False --mosaic=True --mosaic_prob=0.5 --mixup=True --mixup_prob=0.5 --special_aug_ratio=0.7 --label_smoothing=0 --Init_Epoch=0 --Freeze_Epoch=50 --Freeze_batch_size=16 --UnFreeze_Epoch=100 --Unfreeze_batch_size=16 --Freeze_Train=False --Init_lr=0.001 --Min_lr=1e-05 --optimizer_type=adam --momentum=0.9 --weight_decay=0.00

In [ ]:
import torch

print(f"CUDA 사용 가능 여부: {torch.cuda.is_available()}")
print(f"CUDA 디바이스 수: {torch.cuda.device_count()}")
print(f"현재 사용 디바이스: {torch.cuda.current_device()}")
print(f"디바이스 이름: {torch.cuda.get_device_name(0)}")
print(f"PyTorch CUDA 버전: {torch.version.cuda}")
print(f"PyTorch 버전: {torch.__version__}")


CUDA 사용 가능 여부: True
CUDA 디바이스 수: 1
현재 사용 디바이스: 0
디바이스 이름: NVIDIA GeForce RTX 5060 Ti
PyTorch CUDA 버전: 12.8
PyTorch 버전: 2.8.0.dev20250420+cu128
